In [3]:
#Root 경로로 이동
%cd ..


d:\MyGit\BrainMR_MCI


In [4]:
from model import generate_model
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
#from loss import ConRegGroupLoss
import dataloader
import utils
import pandas as pd
from torch.utils.data import DataLoader
from models import resnet
from model import generate_model


In [5]:
config = utils.load_config()
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
X_train,X_test,y_train,y_test = dataloader.dataset_split(df_dataset)
#y_train = y_train.str.replace('MCI','1').replace('CN','0').replace('AD','2')
#y_test = y_test.str.replace('MCI','1').replace('CN','0').replace('AD','2')

In [6]:
traindata=dataloader.MRIDataset(X_train,y_train)
testdata=dataloader.MRIDataset(X_test,y_test)

train_dataloader = DataLoader(traindata, batch_size=4, shuffle=False)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

In [7]:
print('X_train : ',len(X_train))
print('X_test : ',len(X_test))

X_train :  792
X_test :  199


In [8]:
model, _ = generate_model(model_name='resnet',model_depth = 50,n_classes=3,resnet_shortcut='B')

d:\MyGit\BrainMR_MCI\models\resnet.py:143: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [9]:
device = torch.device('cpu')

#model, parameters = generate_model(opt)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr= 0.0001, betas=(0.5, 0.999))
criterion_clf = nn.CrossEntropyLoss().to(device)

In [10]:
epoch = 0
i= 0

model.train(True)
for inputs,_, labels in train_dataloader:
    i+=1
    inputs, labels = inputs.to(device), labels.to(device)
    model.zero_grad()
    clf_output = model(inputs)
    break
    clf_loss = criterion_clf(clf_output, labels)
    print('loss :',clf_loss)
    clf_loss.backward()
    optimizer.step()



In [18]:
len(clf_output)

4

In [19]:
for j in range(len(clf_output)):
    print(1)
    #print(clf_output[j].data.to(device))

1
1
1
1


In [1]:
clf_output

NameError: name 'clf_output' is not defined

In [ ]:
def train_model_epoch(self):
    self.model.train(True)
    for inputs, labels, demors in self.train_dataloader:
        inputs, labels, demors = inputs.cuda(), labels.cuda(), demors.cuda()
        self.model.zero_grad()
        loss = torch.tensor(0.0, requires_grad=True).cuda()
        clf_output, reg_output, per_loss = self.model(inputs)
        clf_loss = self.criterion_clf(clf_output, labels)
        con_reg_group_loss = self.get_con_reg_group_loss.apply(reg_output, demors, self.frequency_dict, labels)
        loss = loss + clf_loss + reg_loss + torch.mean(con_reg_group_loss) + torch.mean(per_loss)
        loss.backward()
        self.optimizer.step()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr= 0.0001, betas=(0.5, 0.999))

criterion_clf = nn.CrossEntropyLoss(weight=torch.Tensor([1, 3])).to('mps')
criterion_reg = nn.SmoothL1Loss(reduction='mean').to('mps')

optimal_valid_matrix = [[0, 0], [0, 0]]
optimal_valid_metric = 0
optimal_epoch = 0

epoch = 0
train_epoch = 300

while epoch < train_epoch:
    self.train_model_epoch()

    print('{}th epoch validation confusion matrix:'.format(self.epoch), valid_matrix)
    print('eval_metric:', "%.4f" % self.eval_metric(valid_matrix), 'and mean squared error ', valid_mse)
    self.save_checkpoint(valid_matrix, valid_mse)
    self.epoch += 1
